In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots

# Load all datasets
users = pd.read_csv("/content/users.csv")
orders = pd.read_csv("/content/orders.csv")
order_items = pd.read_csv("/content/order_items.csv")
products = pd.read_csv("/content/products.csv")
inventory = pd.read_csv("/content/inventory_items.csv")

# Dataset Inspection/Dataset Overview

In [ ]:
for name, df in [
    ("users", users),
    ("orders", orders),
    ("order_items", order_items),
    ("products", products),
    ("inventory", inventory)
]:
  print(f"======= {name} =======")
  print(name, df.info())
  print(f"\nMissing values in {name}:")
  print(df.isnull().sum())

======= users =======
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 15 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   id              100000 non-null  int64  
 1   first_name      100000 non-null  object 
 2   last_name       100000 non-null  object 
 3   email           100000 non-null  object 
 4   age             100000 non-null  int64  
 5   gender          100000 non-null  object 
 6   state           100000 non-null  object 
 7   street_address  100000 non-null  object 
 8   postal_code     100000 non-null  object 
 9   city            99042 non-null   object 
 10  country         100000 non-null  object 
 11  latitude        100000 non-null  float64
 12  longitude       100000 non-null  float64
 13  traffic_source  100000 non-null  object 
 14  created_at      100000 non-null  object 
dtypes: float64(2), int64(2), object(11)
memory usage: 11.4+ MB
users None

Missing va

In [ ]:
print(f"Duplicates:")
print({
    "users": users.duplicated().sum(),
    "orders": orders.duplicated().sum(),
    "order_items": order_items.duplicated().sum(),
    "products": products.duplicated().sum(),
    "inventory": inventory.duplicated().sum()
})

Duplicates:
{'users': np.int64(0), 'orders': np.int64(0), 'order_items': np.int64(0), 'products': np.int64(0), 'inventory': np.int64(0)}


**📑 Data Dictionary**

===== inventory_items =====

|Column|Description|
|------|-----------|
|id| Unique identifier for each inventory item.
|product_id| Identifier for the associated product.
|created_at| Timestamp indicating when the inventory item was created.
|sold_at| Timestamp indicating when the item was sold.
|cost| Cost of the inventory item.
|product_category| Category of the associated product.
|product_name| Name of the associated product.
|product_brand| Brand of the associated product.
|product_retail_price| Retail price of the associated product.
|product_department| Department to which the product belongs.
|product_sku| Stock Keeping Unit (SKU) of the product.
|product_distribution_center_id| Identifier for the distribution center associated with the product.

===== order_items =====

|Column|Description|
|------|-----------|
|id| Unique identifier for each order item.
|order_id| Identifier for the associated order.
|user_id| Identifier for the user who placed the order.
|product_id| Identifier for the associated product.
|inventory_item_id| Identifier for the associated inventory item.
|status| Status of the order item.
|created_at| Timestamp indicating when the order item was created.
|shipped_at| Timestamp indicating when the order item was shipped.
|delivered_at| Timestamp indicating when the order item was delivered.
|returned_at| Timestamp indicating when the order item was returned.

===== orders =====

|Column|Description|
|------|-----------|
|order_id| Unique identifier for each order.
|user_id| Identifier for the user who placed the order.
|status| Status of the order.
|gender| Gender information of the user.
|created_at| Timestamp indicating when the order was created.
|returned_at| Timestamp indicating when the order was returned.
|shipped_at| Timestamp indicating when the order was shipped.
|delivered_at| Timestamp indicating when the order was delivered.
|num_of_item| Number of items in the order.

===== products =====

|Column|Description|
|------|-----------|
|id| Unique identifier for each product.
|cost| Cost of the product.
|category| Category to which the product belongs.
|name| Name of the product.
|brand| Brand of the product.
|retail_price| Retail price of the product.
|department| Department to which the product belongs.
|sku| Stock Keeping Unit (SKU) of the product.
|distribution_center_id| Identifier for the distribution center associated with the product.

===== users =====

|Column|Description|
|------|-----------|
|id| Unique identifier for each user.
|first_name| First name of the user.
|last_name| Last name of the user.
|email| Email address of the user.
|age| Age of the user.
|gender| Gender of the user.
|state| State where the user is located.
|street_address| Street address of the user.
|postal_code| Postal code of the user.
|city| City where the user is located.
|country| Country where the user is located.
|latitude| Latitude coordinate of the user.
|longitude| Longitude coordinate of the user.
|traffic_source| Source of the traffic leading to the user.
|created_at| Timestamp indicating when the user account was created.

For this analysis, I used 5 out of 7 tables from the Looker Ecommerce BigQuery Dataset (Kaggle).

These tables are:

1.	users
  - Contains 100,000 unique customers.
  -	Includes demographic and location fields (age, gender, country, coordinates), which allow segmentation and market-size analysis.
2. orders
    -  Contains 125,226 unique orders, each tied to a user.
    - Includes timestamps for order creation, shipping, delivery, and returns.
    -  Related to:
        - users via `user_id`
        - order_items via `order_id`
3.	order_items
  -  Contains 181,759 order item records, representing line items per order.
  -  Includes orders detail and operational timestamps.
  -  Related to:
      - orders via `order_id`
      - products via `product_id`
      - inventory_items via `inventory_item_id`
4.	products
  -  Contains 29,120 products with retail price, brand, category, etc.
5.	inventory_items
  -  Contains 270,840 inventory records representing stocked items
  -  Provides cost and retail price, enabling profit margin calculations.
> These tables provide enough coverage for key business metrics such as Market Size, Growth, Sales, Revenue, Profitability, and Customer Behavior, making them suitable for performance analysis.
- There are no duplicates detected in any of the 5 tables. However, there are so many missing values thus require imputation or filtering depending on the objective.
  -	orders `returned_at`: 112,696 missing
  -	orders `shipped_at`: 43,765 missing
  -	orders `delivered_at`: 81,342 missing
  -	order_items `shipped_at`: 63,478 missing
  -	order_items `delivered_at`: 117,918 missing
  -	order_items `returned_at`: 163,527 missing
  - Inventory has 170,519 missing `sold_at`, meaning many items are unsold.
-	Timestamp columns (`created_at`, `shipped_at`, `delivered_at`, `returned_at`) should be converted to datetime format.


# Objective/Problem Statements

In [ ]:
orders['status'].value_counts()

,count
status,
Shipped,37577
Complete,31354
Processing,25156
Cancelled,18609
Returned,12530


In [ ]:
order_items.columns

Index(['id', 'order_id', 'user_id', 'product_id', 'inventory_item_id',
       'status', 'created_at', 'shipped_at', 'delivered_at', 'returned_at',
       'sale_price'],
      dtype='object')

In [ ]:
order_items.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 181759 entries, 0 to 181758
Data columns (total 11 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   id                 181759 non-null  int64  
 1   order_id           181759 non-null  int64  
 2   user_id            181759 non-null  int64  
 3   product_id         181759 non-null  int64  
 4   inventory_item_id  181759 non-null  int64  
 5   status             181759 non-null  object 
 6   created_at         181759 non-null  object 
 7   shipped_at         118281 non-null  object 
 8   delivered_at       63841 non-null   object 
 9   returned_at        18232 non-null   object 
 10  sale_price         181759 non-null  float64
dtypes: float64(1), int64(5), object(5)
memory usage: 15.3+ MB


In [ ]:
# Merge order_items with inventory_items
oi_inv = order_items.merge(
    inventory[['id','product_retail_price','cost']],
    left_on='inventory_item_id',
    right_on='id',
    how='left'
)

# Merge with orders
oi_orders = oi_inv.merge(
    orders[['order_id','user_id','status','delivered_at','returned_at']],
    on='order_id',
    how='left'
)

# Merge with users
df = oi_orders.merge(
    users[['id', 'country']],
    left_on='user_id_y',
    right_on='id',
    how='left'
)

# FILTER: only complete orders
df_clean = df[df['status_y'] == 'Complete'].copy()

# Add revenue
df_clean['revenue'] = df_clean['sale_price']

# Actual revenue per country
actual_rev = df_clean.groupby('country')['revenue'].sum().reset_index()

# Unique buyers
unique_buyers = df_clean['user_id_y'].nunique()

global_ARPU = df_clean['revenue'].sum() / unique_buyers
print("Global ARPU:", global_ARPU)

# Potential users per country (TAM)
user_count = users.groupby('country')['id'].nunique().reset_index()
user_count.columns = ['country','num_users']

user_count['potential_revenue'] = user_count['num_users'] * global_ARPU

market = user_count.merge(actual_rev, on='country', how='left')
market['revenue'] = market['revenue'].fillna(0)

market['market_gap'] = market['potential_revenue'] - market['revenue']

# Map
fig = px.choropleth(
    market,
    locations='country',
    locationmode='country names',
    color='market_gap',
    hover_name='country',
    hover_data=['num_users','potential_revenue','revenue'],
    color_continuous_scale=px.colors.sequential.Bluyl,
    height=650,
    title='Market Gap (TAM – Actual Revenue)'
)

fig.show()

Global ARPU: 98.03211721009836


In [ ]:
# MERGE ORDER ITEMS → PRODUCTS
orders_merged = pd.merge(
    order_items,
    products,
    left_on='product_id',
    right_on='id',
    how='left'
)
# Profit calculation
orders_merged['profit'] = orders_merged['sale_price'] - orders_merged['cost']

# FORMAT DATE COLUMNS
date_cols = ['created_at', 'shipped_at', 'delivered_at']
orders_merged[date_cols] = orders_merged[date_cols].apply(
    lambda col: pd.to_datetime(col, errors='coerce', format='mixed')
)

# Filter completed delivered orders with no return
orders_complete = orders_merged.loc[orders_merged['status'] == 'Complete'].copy()

# MERGE TO GET COUNTRY
# orders → users → get country
orders_with_country = orders_complete.merge(
    users[['id', 'country']],
    left_on='user_id',
    right_on='id',
    how='left'
)
# Drop duplicate ID column
orders_with_country = orders_with_country.drop(columns=['id'])

# AGGREGATE BY COUNTRY
country_agg = orders_with_country.groupby('country').agg(
    revenue=('sale_price', 'sum'),
    cost=('cost', 'sum'),
    profit=('profit', 'sum'),
    number_of_orders=('order_id', 'nunique')
).reset_index()

# PLOTTING MAP
fig = px.choropleth(
    country_agg,
    locations='country',
    locationmode='country names',
    color='profit',
    hover_name='country',
    hover_data=['number_of_orders', 'revenue'],
    color_continuous_scale=px.colors.sequential.Bluyl,
    height=650
)

fig.update_layout(title_text='Global Map of Performance (Profit) by Country')
fig.show()

This e-commerce company operates across multiple countries, but each market shows different levels of performance and growth. These variations haven’t been analyzed systematically, making it difficult for the company to identify high-potential markets, detect underperforming regions, and understand what factors actually drive revenue. A structured analysis is needed to uncover these patterns and support global expansion decisions. This project aims to uncover those patterns and provide data-driven guidance for global expansion decisions.

**Objectives:**
- Segment global markets based on revenue performance and growth.
- Identify and prioritize countries for expansion, optimization, or improvement.
- Analyze product and customer-level factors to understand what drives growth in top-performing markets.
- Provide actionable recommendations for global strategy.

**Business Questions:**
- Which countries contribute the most/least to total revenue, profit, and order volume, and how does their performance compare to the global average?
- How is YoY Growth across countries and which markets show the strongest growth or signs of decline?
- Which brands and product categories drive performance in each country, and how do top-performing markets differ in the products they sell?
- How do customer behaviors vary by country?
- Which countries should be prioritized for expansion or optimization?

# Data Preparation

## Create Master Table

In [ ]:
# Merge order_items → inventory_items
df = order_items.merge(
    inventory,
    left_on='inventory_item_id',
    right_on='id',
    how='left',
    suffixes=('', '_inventory')
)

# Merge order_items → products
df = df.merge(
    products,
    left_on='product_id',
    right_on='id',
    how='left',
    suffixes=('', '_product')
)

# Merge order_items → orders
df = df.merge(
    orders,
    left_on='order_id',
    right_on='order_id',
    how='left',
    suffixes=('', '_order')
)

# Merge order_items → users
df = df.merge(
    users,
    left_on='user_id',
    right_on='id',
    how='left',
    suffixes=('', '_user')
)

df.columns

Index(['id', 'order_id', 'user_id', 'product_id', 'inventory_item_id',
       'status', 'created_at', 'shipped_at', 'delivered_at', 'returned_at',
       'sale_price', 'id_inventory', 'product_id_inventory',
       'created_at_inventory', 'sold_at', 'cost', 'product_category',
       'product_name', 'product_brand', 'product_retail_price',
       'product_department', 'product_sku', 'product_distribution_center_id',
       'id_product', 'cost_product', 'category', 'name', 'brand',
       'retail_price', 'department', 'sku', 'distribution_center_id',
       'user_id_order', 'status_order', 'gender', 'created_at_order',
       'returned_at_order', 'shipped_at_order', 'delivered_at_order',
       'num_of_item', 'id_user', 'first_name', 'last_name', 'email', 'age',
       'gender_user', 'state', 'street_address', 'postal_code', 'city',
       'country', 'latitude', 'longitude', 'traffic_source',
       'created_at_user'],
      dtype='object')

In [ ]:
duplicate = [
    'cost', 'cost_product', 'product_category',	'category', 'product_name',	'name', 'product_brand',	'brand',
    'product_retail_price',	'retail_price', 'sale_price', 'product_department',	'department', 'product_sku',	'sku',
    'product_distribution_center_id', 'distribution_center_id', 'created_at',	'created_at_order', 'shipped_at',
    'shipped_at_order', 'delivered_at',	'delivered_at_order', 'returned_at',	'returned_at_order', 'user_id',
    'user_id_order', 'gender',	'gender_user', 'product_id_inventory', 'id'
]

df_duplicate = df[duplicate]
df_duplicate.head()

,cost,cost_product,product_category,category,product_name,name,product_brand,brand,product_retail_price,retail_price,...,delivered_at,delivered_at_order,returned_at,returned_at_order,user_id,user_id_order,gender,gender_user,product_id_inventory,id
0,0.0083,0.0083,Accessories,Accessories,Indestructable Aluminum Aluma Wallet - RED,Indestructable Aluminum Aluma Wallet - RED,marshal,marshal,0.02,0.02,...,NaN,NaN,NaN,NaN,83582,83582,F,F,14235,152013
1,0.0083,0.0083,Accessories,Accessories,Indestructable Aluminum Aluma Wallet - RED,Indestructable Aluminum Aluma Wallet - RED,marshal,marshal,0.02,0.02,...,2023-03-18 01:08:00+00:00,2023-03-18 01:08:00+00:00,NaN,NaN,22551,22551,F,F,14235,40993
2,0.0083,0.0083,Accessories,Accessories,Indestructable Aluminum Aluma Wallet - RED,Indestructable Aluminum Aluma Wallet - RED,marshal,marshal,0.02,0.02,...,2023-12-10 10:04:00+00:00,2023-12-10 10:04:00+00:00,NaN,NaN,28215,28215,F,F,14235,51224
3,0.0083,0.0083,Accessories,Accessories,Indestructable Aluminum Aluma Wallet - RED,Indestructable Aluminum Aluma Wallet - RED,marshal,marshal,0.02,0.02,...,NaN,NaN,NaN,NaN,20165,20165,F,F,14235,36717
4,0.0083,0.0083,Accessories,Accessories,Indestructable Aluminum Aluma Wallet - RED,Indestructable Aluminum Aluma Wallet - RED,marshal,marshal,0.02,0.02,...,NaN,NaN,NaN,NaN,71954,71954,F,F,14235,131061


In [ ]:
df_used = df[[
    # item-level
    'order_id', 'user_id', 'product_id', 'inventory_item_id', 'num_of_item', 'cost', 'sale_price',

    # product info
    'category', 'product_name', 'brand', 'department', 'product_retail_price',

    # order timestamps/status
    'status', 'created_at', 'shipped_at', 'delivered_at', 'returned_at',

    # users
    'first_name', 'last_name', 'email', 'age', 'gender', 'country', 'city', 'state', 'traffic_source'
]]
df_used.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 181759 entries, 0 to 181758
Data columns (total 26 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   order_id              181759 non-null  int64  
 1   user_id               181759 non-null  int64  
 2   product_id            181759 non-null  int64  
 3   inventory_item_id     181759 non-null  int64  
 4   num_of_item           181759 non-null  int64  
 5   cost                  181759 non-null  float64
 6   sale_price            181759 non-null  float64
 7   category              181759 non-null  object 
 8   product_name          181749 non-null  object 
 9   brand                 181610 non-null  object 
 10  department            181759 non-null  object 
 11  product_retail_price  181759 non-null  float64
 12  status                181759 non-null  object 
 13  created_at            181759 non-null  object 
 14  shipped_at            118281 non-null  object 
 15  

In [ ]:
# Validate
print("Rows:", len(df_used))
print("Columns:", len(df_used.columns))
df_used.head()

Rows: 181759
Columns: 26


,order_id,user_id,product_id,inventory_item_id,num_of_item,cost,sale_price,category,product_name,brand,...,returned_at,first_name,last_name,email,age,gender,country,city,state,traffic_source
0,104663,83582,14235,410368,1,0.0083,0.02,Accessories,Indestructable Aluminum Aluma Wallet - RED,marshal,...,NaN,Kathryn,Ortega,kathrynortega@example.net,29,F,China,Jinhua,Hebei,Search
1,28204,22551,14235,110590,1,0.0083,0.02,Accessories,Indestructable Aluminum Aluma Wallet - RED,marshal,...,NaN,Terri,Alvarez,terrialvarez@example.net,35,F,China,Yulin,Jiangsu,Organic
2,35223,28215,14235,138236,4,0.0083,0.02,Accessories,Indestructable Aluminum Aluma Wallet - RED,marshal,...,NaN,Rebecca,Mcgee,rebeccamcgee@example.org,49,F,United States,Longview,Texas,Organic
3,25278,20165,14235,99072,4,0.0083,0.02,Accessories,Indestructable Aluminum Aluma Wallet - RED,marshal,...,NaN,Shelly,Rogers,shellyrogers@example.net,44,F,Japan,Kakogawa City,Hyogo,Search
4,90241,71954,14235,353798,3,0.0083,0.02,Accessories,Indestructable Aluminum Aluma Wallet - RED,marshal,...,NaN,Mariah,Stephens,mariahstephens@example.com,59,F,United Kingdom,Lancaster,England,Search


I created marged table containing 181,759 observations, combining fields from orders, order_items, products, inventory_items, and users. This table consolidates transaction details, product attributes, order operational timestamps, and customer demographics, enabling comprehensive analysis for global revenue performance, market growth, country comparisons, and customer behavior.

## Data Cleaning

### Data Inspection

In [ ]:
# Check format
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 181759 entries, 0 to 181758
Data columns (total 55 columns):
 #   Column                          Non-Null Count   Dtype  
---  ------                          --------------   -----  
 0   id                              181759 non-null  int64  
 1   order_id                        181759 non-null  int64  
 2   user_id                         181759 non-null  int64  
 3   product_id                      181759 non-null  int64  
 4   inventory_item_id               181759 non-null  int64  
 5   status                          181759 non-null  object 
 6   created_at                      181759 non-null  object 
 7   shipped_at                      118281 non-null  object 
 8   delivered_at                    63841 non-null   object 
 9   returned_at                     18232 non-null   object 
 10  sale_price                      181759 non-null  float64
 11  id_inventory                    181759 non-null  int64  
 12  product_id_inven

In [ ]:
# Check duplicates
df_used.duplicated().sum()

np.int64(0)

In [ ]:
# Check missing value
df_used.isnull().sum()

,0
order_id,0
user_id,0
product_id,0
inventory_item_id,0
num_of_item,0
cost,0
sale_price,0
category,0
product_name,10
brand,149


In [ ]:
# Missing value perentage
product_name_missing = 100 * len(df_used[df_used['product_name'].isnull()]) / len(df_used)
brand_missing = 100 * len(df_used[df_used['brand'].isnull()]) / len(df_used)
shipped_at_missing = 100 * len(df_used[df_used['shipped_at'].isnull()]) / len(df_used)
delivered_at_missing = 100 * len(df_used[df_used['delivered_at'].isnull()]) / len(df_used)
returned_at_missing = 100 * len(df_used[df_used['returned_at'].isnull()]) / len(df_used)
city = 100 * len(df_used[df_used['city'].isnull()]) / len(df_used)

print(f"Missing product_name: {product_name_missing:.2f}%")
print(f"Missing brand: {brand_missing:.2f}%")
print(f"Missing shipped_at: {shipped_at_missing:.2f}%")
print(f"Missing delivered_at: {delivered_at_missing:.2f}%")
print(f"Missing returned_at: {returned_at_missing:.2f}%")
print(f"Missing city: {city:.2f}%")

Missing product_name: 0.01%
Missing brand: 0.08%
Missing shipped_at: 34.92%
Missing delivered_at: 64.88%
Missing returned_at: 89.97%
Missing city: 0.92%


In [ ]:
df_used.describe()

,order_id,user_id,product_id,inventory_item_id,num_of_item,cost,sale_price,product_retail_price,age
count,181759.000000,181759.000000,181759.000000,181759.000000,181759.000000,181759.000000,181759.000000,181759.000000,181759.000000
mean,62579.982609,49983.272020,15254.151838,245291.860007,1.901457,28.655281,59.568544,59.568544,41.063749
std,36125.987285,28830.895739,8413.482833,141661.912894,1.066470,31.022340,66.657262,66.657262,17.032648
min,1.000000,1.000000,1.000000,3.000000,1.000000,0.008300,0.020000,0.020000,12.000000
25%,31266.500000,25031.500000,7979.500000,122565.000000,1.000000,11.366210,24.500000,24.500000,26.000000
50%,62529.000000,49983.000000,15966.000000,245284.000000,2.000000,19.829000,39.990002,39.990002,41.000000
75%,93845.500000,74867.500000,22504.000000,367983.000000,2.000000,34.545320,69.949997,69.949997,56.000000
max,125226.000000,100000.000000,29120.000000,490705.000000,4.000000,557.151002,999.000000,999.000000,70.000000


In [ ]:
cat_cols = df_used.select_dtypes(include='object').columns

for col in cat_cols:
  print(f"======= {col} =======")
  display(df_used[col].value_counts())
  print()

======= category =======


,count
category,
Intimates,13474
Jeans,12698
Tops & Tees,11925
Fashion Hoodies & Sweatshirts,11885
Swim,11428
Sweaters,11265
Shorts,11038
Sleep & Lounge,11002
Accessories,9753



======= product_name =======


,count
product_name,
Wrangler Men's Premium Performance Cowboy Cut Jean,62
Puma Men's Socks,48
7 For All Mankind Men's Standard Classic Straight Leg Jean,41
True Religion Men's Ricky Straight Jean,37
Kenneth Cole Men's Straight Leg Jean,36
...,...
Thorlo Mens Dress Crew Sock,1
ASCIS Hydrology Low Socks (Pack of 3),1
Hanes Women's Control Top Reinforced Toe Silk Reflections Panty Hose,1



======= brand =======


...

,count
city,
Shanghai,4608
Beijing,4083
Seoul,2704
Shenzhen,2484
Dongguan,1716
...,...
Straelen,1
Teralba,1
Whiteman,1



======= state =======


,count
state,
Guangdong,9719
England,7395
California,6656
Shanghai,4520
Texas,4372
...,...
Nagasaki,5
Shimane,4
Kagawa,3



======= traffic_source =======


,count
traffic_source,
Search,127277
Organic,27535
Facebook,10508
Email,9086
Display,7353


**Observations:**
- There were no duplicates found across the combined dataset. There were also no structural anomalies.
- Missing values exist mainly in timestamp fields and a few minor categorical columns:
  - product_name: 0.01% -> try hanlidng with product_id or else fill with `unknown`.
  - brand: 0.08% -> try handling using product_id or else fill with `unknown`.
  - city: 0.92% -> drop because state and country already provide sufficient geographic granularity for the required business questions.
  - shipped_at: 34.92% -> drop
  - delivered_at: 64.88% -> drop
  - returned_at: 89.97% -> missing values means order was not returned so convert to **boolean**.
- There were outliers in price and cost but appear operationally realistic, so not require handling/removal.
- To support the business questions, additional engineered columns should such as revenue (sale_price * num_of_item), profit (revenue - cost * num_of_item), retail markup for comparing pricing strategy across countries/brands ((sale_price - cost) / cost), and customer type (New vs Returning).

### Data Handling/Feature Engineering

In [ ]:
df_clean = df_used.copy()

In [ ]:
# Drop city, shipped_at, and delivered_at
df_clean.drop(columns=['city', 'shipped_at', 'delivered_at'], inplace=True)

In [ ]:
# product_name and handling -> fill by most common per product_id
for col in ['product_name', 'brand']:
    df_clean[col] = df_clean.groupby('product_id')[col]\
                .transform(lambda x: x.fillna(x.mode().iloc[0] if not x.mode().empty else "Unknown"))

In [ ]:
df_clean[(df_clean['product_name'] == 'Unknown') | (df_clean['brand'] == 'Unknown')]

,order_id,user_id,product_id,inventory_item_id,num_of_item,cost,sale_price,category,product_name,brand,...,created_at,returned_at,first_name,last_name,email,age,gender,country,state,traffic_source
599,109450,87443,13921,429063,1,1.641900,4.210000,Accessories,NEW Aluminum Credit Card Wallet - RFID Blockin...,Unknown,...,2022-12-19 10:39:33+00:00,NaN,Veronica,Perez,veronicaperez@example.net,20,F,China,Henan,Search
600,17324,13803,15757,67585,1,2.235510,4.210000,Plus,NEW Aluminum Credit Card Wallet - RFID Blockin...,Unknown,...,2023-05-25 06:54:13+00:00,NaN,Peggy,Bryant,peggybryant@example.com,62,F,Brasil,Espírito Santo,Facebook
601,61976,49531,13921,243102,1,1.641900,4.210000,Accessories,NEW Aluminum Credit Card Wallet - RFID Blockin...,Unknown,...,2023-08-08 04:18:02+00:00,NaN,Olivia,Riley,oliviariley@example.com,35,F,Spain,Cataluña,Search
602,108837,86958,15757,426762,1,2.235510,4.210000,Plus,NEW Aluminum Credit Card Wallet - RFID Blockin...,Unknown,...,2022-08-15 15:34:12+00:00,NaN,Darlene,Martinez,darlenemartinez@example.com,40,F,Spain,Comunidad Valenciana,Search
603,111145,88775,15757,435613,2,2.235510,4.210000,Plus,NEW Aluminum Credit Card Wallet - RFID Blockin...,Unknown,...,2023-02-05 04:25:10+00:00,NaN,Michelle,Ingram,michelleingram@example.net,14,F,United States,Oklahoma,Facebook
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
167997,41245,33002,24455,161626,2,67.335453,147.990005,Outerwear & Coats,Unknown,Tru-Spec,...,2024-01-19 13:59:09+00:00,NaN,Ronald,Carroll,ronaldcarroll@example.net,29,M,China,Shanghai,Search
167998,81717,65169,24455,320162,1,67.335453,147.990005,Outerwear & Coats,Unknown,Tru-Spec,...,2022-11-02 11:32:38+00:00,NaN,James,Brown,jamesbrown@example.net,21,M,France,Occitanie,Search
168009,17414,13859,24455,67924,2,67.335453,147.990005,Outerwear & Coats,Unknown,Tru-Spec,...,2023-10-19 22:53:29+00:00,NaN,Jason,Jones,jasonjones@example.net,53,M,South Korea,Seoul,Search
168012,90848,72450,24455,356297,1,67.335453,147.990005,Outerwear & Coats,Unknown,Tru-Spec,...,2023-11-02 15:09:31+00:00,NaN,Michael,Price,michaelprice@example.net,67,M,Spain,Andalucía,Organic


In [ ]:
# returned_at convert to NaT + derive a boolean
df_clean['is_returned'] = df_clean['returned_at'].notna()
df_clean.drop(columns=['returned_at'], inplace=True)

In [ ]:
# Add column
df_clean['revenue'] = df_clean['sale_price'] * df_clean['num_of_item']
df_clean['profit'] = (df_clean['sale_price'] - df_clean['cost']) * df_clean['num_of_item']
df_clean['retail_markup_pct'] = ((df_clean['sale_price'] - df_clean['cost']) / df_clean['cost']) * 100
df_clean = df_clean.sort_values(['user_id', 'created_at'])
df_clean['order_number'] = df_clean.groupby('user_id').cumcount() + 1
df_clean['customer_type'] = df_clean['order_number'].apply(lambda x: 'New' if x == 1 else 'Returning')

df_clean.head()

,order_id,user_id,product_id,inventory_item_id,num_of_item,cost,sale_price,category,product_name,brand,...,gender,country,state,traffic_source,is_returned,revenue,profit,retail_markup_pct,order_number,customer_type
35284,1,1,7656,9,3,8.375810,19.990000,Blazers & Jackets,Plus size Button Closure Cropped Bolero Jacket...,eVogues Apparel,...,F,South Korea,Gyeonggi-do,Search,False,59.969999,34.842569,138.663483,1,New
22628,1,1,2953,3,3,6.000000,15.000000,Active,Tommy Hilfiger Women's 3 Pack Sports Crew,Tommy Hilfiger,...,F,South Korea,Gyeonggi-do,Search,False,45.000000,27.000000,149.999999,2,Returning
163080,1,1,4731,7,3,66.750000,125.000000,Jeans,Joe's Jeans Women's Skinny Jeans,Joe's Jeans,...,F,South Korea,Gyeonggi-do,Search,False,375.000000,174.750000,87.265918,3,Returning
39779,2,2,25774,11,1,10.384000,22.000000,Underwear,Tommy Bahama Tropical Print Boxer Short,Tommy Bahama,...,M,Brasil,São Paulo,Organic,False,22.000000,11.616000,111.864406,1,New
159008,5,3,18177,18,1,49.825469,109.989998,Active,Smartwool Men's Midweight Bottom,SmartWool,...,M,United States,Florida,Organic,False,109.989998,60.164529,120.750551,1,New


In [ ]:
df_clean.info()

<class 'pandas.core.frame.DataFrame'>
Index: 181759 entries, 35284 to 41027
Data columns (total 28 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   order_id              181759 non-null  int64  
 1   user_id               181759 non-null  int64  
 2   product_id            181759 non-null  int64  
 3   inventory_item_id     181759 non-null  int64  
 4   num_of_item           181759 non-null  int64  
 5   cost                  181759 non-null  float64
 6   sale_price            181759 non-null  float64
 7   category              181759 non-null  object 
 8   product_name          181759 non-null  object 
 9   brand                 181759 non-null  object 
 10  department            181759 non-null  object 
 11  product_retail_price  181759 non-null  float64
 12  status                181759 non-null  object 
 13  created_at            181759 non-null  object 
 14  first_name            181759 non-null  object 
 15  la

In [ ]:
# Convert CSV
df_clean.to_csv('performance_analysis_ecommerce_expansion.csv', index=False)

In [ ]:
# Convesrt to Excel
df_clean.to_excel('performance_analysis_ecommerce_expansion.xlsx', index=False)